# TLSH 測試工作坊 / TLSH Testing Workshop

這個 notebook 展示如何使用我們的 TLSH 分析工具進行兩種不同的測試案例  
This notebook demonstrates how to use our TLSH analysis tool for two different test cases

## 學習目標 / Learning Objectives:
1. **企業應用測試** / **Enterprise Application Testing**: 使用 TLSH 進行資料外洩檢測 / Use TLSH for data leak detection
2. **大規模資料分群** / **Large-scale Data Clustering**: 對惡意軟體資料集進行分群分析 / Cluster analysis on malware datasets
3. **Python 工具整合** / **Python Tool Integration**: 結合 Python 腳本與 pytest 測試 / Integrate Python scripts with pytest testing
4. **跨語言驗證** / **Cross-language Validation**: 比較 Python 與 Golang 實作結果 / Compare Python and Golang implementation results

## 環境設置 / Environment Setup

In [ ]:
# 導入必要的函式庫 / Import required libraries
import sys
import os
import json
import subprocess
import pandas as pd
import matplotlib.pyplot as plt

# 添加當前目錄到路徑 / Add current directory to path
if '.' not in sys.path:
    sys.path.insert(0, '.')

try:
    from tlsh_analyzer import TLSHAnalyzer
    print("✅ 成功導入 TLSH 分析器 / Successfully imported TLSH Analyzer")
except ImportError as e:
    print(f"❌ 導入錯誤 / Import error: {e}")
    print("請確保 tlsh_analyzer.py 存在於當前目錄 / Please ensure tlsh_analyzer.py exists in current directory")

## 案例 1: 企業資料外洩檢測 / Case 1: Enterprise Data Leak Detection

模擬企業環境中的資料外洩檢測情境  
Simulate data leak detection scenarios in enterprise environment

In [ ]:
# 建立測試情境 / Create test scenarios
analyzer = TLSHAnalyzer()
analyzer.verbose = True

print("🏢 企業資料外洩檢測測試 / Enterprise Data Leak Detection Test")
print("=" * 70)

# 情境 1: 生產伺服器上的客戶資料 / Scenario 1: Customer data on production server
production_data = '''
{
    "customer_database": {
        "customers": [
            {
                "id": "CUST_001",
                "name": "王小明",
                "email": "wang.xiaoming@company.com",
                "phone": "+886-2-1234-5678",
                "address": "台北市信義區信義路五段7號101大樓32F",
                "credit_card": "**** **** **** 1234",
                "purchase_history": [
                    {"date": "2024-01-15", "amount": 15000, "product": "企業軟體授權"}
                ]
            },
            {
                "id": "CUST_002",
                "name": "李美華",
                "email": "li.meihua@company.com",
                "phone": "+886-2-8765-4321",
                "address": "新北市板橋區中山路一段161號",
                "credit_card": "**** **** **** 5678",
                "purchase_history": [
                    {"date": "2024-02-10", "amount": 8500, "product": "技術支援包"}
                ]
            }
        ]
    }
}
''' * 2  # 重複內容以滿足 TLSH 最小長度要求 / Repeat content to meet TLSH minimum length requirement

# 情境 2: 測試伺服器上發現的可疑資料 / Scenario 2: Suspicious data found on test server
test_server_data = '''
{
    "customer_database": {
        "customers": [
            {
                "id": "CUST_001",
                "name": "王小明",
                "email": "wang.xiaoming@company.com",
                "phone": "+886-2-1234-5678",
                "address": "台北市信義區信義路五段7號101大樓32F",
                "credit_card": "**** **** **** 1234",
                "purchase_history": [
                    {"date": "2024-01-15", "amount": 15000, "product": "企業軟體授權"}
                ],
                "last_access": "2024-03-15T10:30:00Z"
            },
            {
                "id": "CUST_002",
                "name": "李美華",
                "email": "li.meihua@company.com",
                "phone": "+886-2-8765-4321",
                "address": "新北市板橋區中山路一段161號",
                "credit_card": "**** **** **** 5678",
                "purchase_history": [
                    {"date": "2024-02-10", "amount": 8500, "product": "技術支援包"}
                ],
                "last_access": "2024-03-16T09:15:00Z"
            }
        ]
    }
}
''' * 2

# 執行比較 / Execute comparison
result1 = analyzer.compare_two_texts(production_data, test_server_data)
print(f"\n📊 檢測結果 / Detection Results:")
print(f"距離 / Distance: {result1['distance']}")
print(f"分類 / Classification: {result1['similarity_class']}")
print(f"風險等級 / Risk Level: {result1['risk_level']}")
print(f"\n解釋 / Interpretation:")
print(f"中文: {result1['interpretation']['zh']}")
print(f"English: {result1['interpretation']['en']}")

In [ ]:
# 更多測試案例 / Additional test cases
print("\n🔍 額外的檢測案例 / Additional Detection Cases")
print("=" * 50)

# 案例 A: 格式轉換後的相同資料 / Case A: Same data after format conversion
csv_format_data = '''
customer_id,name,email,phone,address,credit_card,last_purchase_amount,last_purchase_product
CUST_001,王小明,wang.xiaoming@company.com,+886-2-1234-5678,"台北市信義區信義路五段7號101大樓32F",**** **** **** 1234,15000,企業軟體授權
CUST_002,李美華,li.meihua@company.com,+886-2-8765-4321,"新北市板橋區中山路一段161號",**** **** **** 5678,8500,技術支援包
''' * 10  # 重複以滿足長度要求 / Repeat to meet length requirement

result_a = analyzer.compare_two_texts(production_data, csv_format_data)
print(f"\n📄 JSON vs CSV 格式比較 / JSON vs CSV format comparison:")
print(f"距離: {result_a['distance']}, 分類: {result_a['similarity_class']}")

# 案例 B: 完全不同的資料 / Case B: Completely different data
different_data = '''
{
    "employee_database": {
        "employees": [
            {
                "emp_id": "EMP_001",
                "name": "張工程師",
                "department": "研發部",
                "salary": 80000,
                "skills": ["Python", "JavaScript", "Docker"]
            }
        ]
    }
}
''' * 5

result_b = analyzer.compare_two_texts(production_data, different_data)
print(f"\n👥 客戶資料 vs 員工資料比較 / Customer vs Employee data comparison:")
print(f"距離: {result_b['distance']}, 分類: {result_b['similarity_class']}")

# 視覺化結果 / Visualize results
distances = [result1['distance'], result_a['distance'], result_b['distance']]
labels = ['測試伺服器\nTest Server', 'CSV格式\nCSV Format', '員工資料\nEmployee Data']

plt.figure(figsize=(10, 6))
bars = plt.bar(labels, distances, color=['red', 'orange', 'green'], alpha=0.7)
plt.ylabel('TLSH 距離 / TLSH Distance')
plt.title('企業資料外洩檢測結果 / Enterprise Data Leak Detection Results')
plt.axhline(y=50, color='red', linestyle='--', alpha=0.5, label='高風險閾值 / High Risk Threshold')
plt.axhline(y=100, color='orange', linestyle='--', alpha=0.5, label='中風險閾值 / Medium Risk Threshold')

# 在柱狀圖上顯示數值 / Display values on bars
for bar, distance in zip(bars, distances):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5,
             str(distance), ha='center', va='bottom', fontweight='bold')

plt.legend()
plt.tight_layout()
plt.show()

print("\n💡 結論 / Conclusions:")
print(f"• 測試伺服器資料: 距離 {result1['distance']} - 可能是資料外洩")
print(f"• Test server data: Distance {result1['distance']} - Possible data leak")
print(f"• CSV格式資料: 距離 {result_a['distance']} - 相同資料不同格式")
print(f"• CSV format data: Distance {result_a['distance']} - Same data different format")
print(f"• 員工資料: 距離 {result_b['distance']} - 完全不同的資料")
print(f"• Employee data: Distance {result_b['distance']} - Completely different data")

## 案例 2: 大規模資料集分群分析 / Case 2: Large-scale Dataset Clustering Analysis

使用真實的惡意軟體資料集進行 DBSCAN 分群  
Use real malware dataset for DBSCAN clustering

In [ ]:
print("🔬 大規模資料集分群分析 / Large-scale Dataset Clustering Analysis")
print("=" * 70)

# 檢查資料集是否存在 / Check if dataset exists
dataset_path = "./data/mb_1K.csv"
if os.path.exists(dataset_path):
    print(f"✅ 找到資料集 / Found dataset: {dataset_path}")
    
    # 執行分群分析 / Execute clustering analysis
    result2 = analyzer.analyze_file_dataset(dataset_path, eps=30, min_samples=2)
    
    print(f"\n📊 分群結果摘要 / Clustering Results Summary:")
    print(f"總樣本數 / Total samples: {result2['total_samples']}")
    print(f"找到的群集數 / Clusters found: {result2['results']['n_clusters']}")
    print(f"雜訊點數 / Noise points: {result2['results']['n_noise']}")
    print(f"分群效率 / Clustering efficiency: {result2['results']['clustering_efficiency']}%")
    
    if result2['family_info']['has_family_labels']:
        print(f"唯一家族數 / Unique families: {result2['family_info']['unique_families']}")
    
    print(f"\n摘要 / Summary:")
    print(f"中文: {result2['summary']['zh']}")
    print(f"English: {result2['summary']['en']}")
    
    # 測試不同的參數設定 / Test different parameter settings
    print(f"\n🔧 參數調整測試 / Parameter Tuning Tests:")
    
    eps_values = [20, 30, 40, 50]
    results_comparison = []
    
    for eps in eps_values:
        try:
            temp_result = analyzer.analyze_file_dataset(dataset_path, eps=eps, min_samples=2)
            results_comparison.append({
                'eps': eps,
                'clusters': temp_result['results']['n_clusters'],
                'noise': temp_result['results']['n_noise'],
                'efficiency': temp_result['results']['clustering_efficiency']
            })
            print(f"eps={eps}: {temp_result['results']['n_clusters']} 群集, {temp_result['results']['n_noise']} 雜訊點")
        except Exception as e:
            print(f"eps={eps}: 錯誤 / Error - {e}")
    
    # 視覺化參數比較 / Visualize parameter comparison
    if results_comparison:
        df_comparison = pd.DataFrame(results_comparison)
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # 群集數量比較 / Clusters comparison
        ax1.plot(df_comparison['eps'], df_comparison['clusters'], 'bo-', linewidth=2, markersize=8)
        ax1.set_xlabel('eps 參數值 / eps parameter value')
        ax1.set_ylabel('群集數量 / Number of clusters')
        ax1.set_title('eps 參數對群集數量的影響\nEffect of eps parameter on cluster count')
        ax1.grid(True, alpha=0.3)
        
        # 分群效率比較 / Efficiency comparison
        ax2.bar(df_comparison['eps'].astype(str), df_comparison['efficiency'], 
                color='skyblue', alpha=0.7)
        ax2.set_xlabel('eps 參數值 / eps parameter value')
        ax2.set_ylabel('分群效率 (%) / Clustering efficiency (%)')
        ax2.set_title('eps 參數對分群效率的影響\nEffect of eps parameter on clustering efficiency')
        
        # 在柱狀圖上顯示數值 / Display values on bars
        for i, v in enumerate(df_comparison['efficiency']):
            ax2.text(i, v + 1, f'{v:.1f}%', ha='center', va='bottom', fontweight='bold')
        
        plt.tight_layout()
        plt.show()
        
        print(f"\n📈 最佳參數建議 / Optimal Parameter Recommendations:")
        best_efficiency = df_comparison.loc[df_comparison['efficiency'].idxmax()]
        print(f"最高效率: eps={best_efficiency['eps']}, 效率={best_efficiency['efficiency']:.1f}%")
        print(f"Highest efficiency: eps={best_efficiency['eps']}, efficiency={best_efficiency['efficiency']:.1f}%")

else:
    print(f"❌ 找不到資料集 / Dataset not found: {dataset_path}")
    print(f"請確保資料檔案存在 / Please ensure data file exists")
    
    # 使用範例功能 / Use example function
    print(f"\n🔄 使用內建範例 / Using built-in example:")
    example_result = analyzer.run_case_2_example()
    print(json.dumps(example_result, indent=2, ensure_ascii=False))

## Python 工具測試 / Python Tool Testing

使用 pytest 測試我們的 TLSH 分析工具  
Use pytest to test our TLSH analysis tool

In [ ]:
print("🧪 執行 pytest 測試 / Running pytest Tests")
print("=" * 50)

# 檢查測試檔案是否存在 / Check if test file exists
test_file = "test_tlsh_analyzer.py"
if os.path.exists(test_file):
    print(f"✅ 找到測試檔案 / Found test file: {test_file}")
    
    # 執行 pytest / Run pytest
    try:
        result = subprocess.run(
            ["python", "-m", "pytest", test_file, "-v", "--tb=short"],
            capture_output=True,
            text=True,
            timeout=60  # 60秒超時 / 60 second timeout
        )
        
        print(f"\n📊 測試結果 / Test Results:")
        print(f"返回碼 / Return code: {result.returncode}")
        
        if result.stdout:
            print(f"\n標準輸出 / Standard Output:")
            print(result.stdout)
        
        if result.stderr:
            print(f"\n錯誤輸出 / Error Output:")
            print(result.stderr)
        
        if result.returncode == 0:
            print(f"\n✅ 所有測試通過 / All tests passed!")
        else:
            print(f"\n❌ 部分測試失敗 / Some tests failed")
            
    except subprocess.TimeoutExpired:
        print(f"⏱️ 測試超時 / Test timeout after 60 seconds")
    except Exception as e:
        print(f"❌ 執行測試時發生錯誤 / Error running tests: {e}")
        
        # 手動執行部分測試 / Manually run some tests
        print(f"\n🔄 手動執行基本測試 / Manually running basic tests:")
        
        # 測試基本功能 / Test basic functionality
        try:
            test_analyzer = TLSHAnalyzer()
            test_text1 = "這是一個測試文字，用於驗證 TLSH 功能是否正常運作。" * 3
            test_text2 = "這是一個測試文字，用於驗證 TLSH 功能是否正常運作！" * 3
            
            test_result = test_analyzer.compare_two_texts(test_text1, test_text2)
            print(f"✅ 基本文字比較測試通過 / Basic text comparison test passed")
            print(f"   距離 / Distance: {test_result['distance']}")
            
        except Exception as test_error:
            print(f"❌ 基本測試失敗 / Basic test failed: {test_error}")

else:
    print(f"❌ 找不到測試檔案 / Test file not found: {test_file}")

## 指令列工具示範 / Command Line Tool Demonstration

展示如何使用 tlsh_analyzer.py 作為指令列工具  
Demonstrate how to use tlsh_analyzer.py as a command line tool

In [ ]:
print("💻 指令列工具示範 / Command Line Tool Demonstration")
print("=" * 60)

# 檢查主要腳本是否存在 / Check if main script exists
script_file = "tlsh_analyzer.py"
if os.path.exists(script_file):
    print(f"✅ 找到主要腳本 / Found main script: {script_file}")
    
    # 示範不同的指令列用法 / Demonstrate different command line usages
    commands = [
        {
            "name": "案例 1 範例 / Case 1 Example",
            "cmd": ["python", script_file, "case1", "--example", "--verbose"]
        },
        {
            "name": "案例 2 範例 / Case 2 Example", 
            "cmd": ["python", script_file, "case2", "--example", "--verbose"]
        },
        {
            "name": "說明文件 / Help Documentation",
            "cmd": ["python", script_file, "--help"]
        }
    ]
    
    for i, command_info in enumerate(commands, 1):
        print(f"\n🔧 示範 {i}: {command_info['name']}")
        print(f"指令 / Command: {' '.join(command_info['cmd'])}")
        print("-" * 40)
        
        try:
            result = subprocess.run(
                command_info['cmd'],
                capture_output=True,
                text=True,
                timeout=30
            )
            
            if result.returncode == 0:
                print(f"✅ 執行成功 / Execution successful")
                if result.stdout:
                    # 限制輸出長度 / Limit output length
                    output = result.stdout
                    if len(output) > 1000:
                        output = output[:1000] + "\n...（輸出已截斷 / Output truncated）"
                    print(output)
            else:
                print(f"❌ 執行失敗 / Execution failed (返回碼 / Return code: {result.returncode})")
                if result.stderr:
                    print(f"錯誤 / Error: {result.stderr[:500]}")
                    
        except subprocess.TimeoutExpired:
            print(f"⏱️ 指令超時 / Command timeout")
        except Exception as e:
            print(f"❌ 執行錯誤 / Execution error: {e}")

else:
    print(f"❌ 找不到主要腳本 / Main script not found: {script_file}")

print(f"\n📚 使用指南 / Usage Guide:")
print(f"1. 基本文字比較 / Basic text comparison:")
print(f"   python tlsh_analyzer.py case1 --text1 '文字1' --text2 '文字2'")
print(f"\n2. 資料集分群分析 / Dataset clustering analysis:")
print(f"   python tlsh_analyzer.py case2 --csv data/mb_1K.csv")
print(f"\n3. 自訂參數 / Custom parameters:")
print(f"   python tlsh_analyzer.py case2 --csv data.csv --eps 25 --min_samples 3")
print(f"\n4. 保存結果 / Save results:")
print(f"   python tlsh_analyzer.py case1 --example --output results.json")

## 總結和下一步 / Summary and Next Steps

這個工作坊展示了：  
This workshop demonstrated:

### 已完成的功能 / Completed Features:
✅ **Python TLSH 分析器** / **Python TLSH Analyzer**  
✅ **企業資料外洩檢測** / **Enterprise Data Leak Detection**  
✅ **大規模資料集分群** / **Large-scale Dataset Clustering**  
✅ **pytest 測試套件** / **pytest Test Suite**  
✅ **指令列介面** / **Command Line Interface**  

### 下一步 / Next Steps:
🔄 **Golang 版本實作** / **Golang Version Implementation**  
🔄 **跨語言結果驗證** / **Cross-language Result Validation**  
🔄 **效能基準測試** / **Performance Benchmarking**  
🔄 **整合 CI/CD 流程** / **CI/CD Pipeline Integration**  

In [ ]:
print("🎉 TLSH 測試工作坊完成！/ TLSH Testing Workshop Completed!")
print("=" * 60)

# 總結統計 / Summary statistics
print(f"📊 工作坊統計 / Workshop Statistics:")
print(f"• 測試案例數 / Test cases: 2 (企業檢測 + 資料分群 / Enterprise detection + Data clustering)")
print(f"• Python 檔案 / Python files: 2 (tlsh_analyzer.py + test_tlsh_analyzer.py)")
print(f"• 支援的輸入格式 / Supported input formats: JSON, CSV, 純文字 / Plain text")
print(f"• 支援的輸出格式 / Supported output formats: JSON, 控制台 / Console")

print(f"\n🚀 建議的下一步行動 / Suggested Next Actions:")
print(f"1. 實作 Golang 版本來比較效能 / Implement Golang version for performance comparison")
print(f"2. 建立自動化測試流程 / Set up automated testing pipeline")
print(f"3. 整合到企業安全監控系統 / Integrate into enterprise security monitoring system")
print(f"4. 擴展到更多檔案格式支援 / Extend to support more file formats")

print(f"\n💡 學習重點 / Key Learning Points:")
print(f"• TLSH 適合檢測資料相似性而非精確匹配 / TLSH is suitable for detecting data similarity, not exact matching")
print(f"• DBSCAN 參數調整對分群結果有重要影響 / DBSCAN parameter tuning significantly affects clustering results")
print(f"• pytest 提供完整的測試覆蓋和驗證 / pytest provides comprehensive testing coverage and validation")
print(f"• 指令列工具增加了實用性和整合能力 / Command line tool enhances practicality and integration capability")

print(f"\n🎯 實際應用場景 / Real-world Application Scenarios:")
print(f"• 資安事件調查 / Security incident investigation")
print(f"• 資料外洩檢測 / Data leak detection")
print(f"• 惡意軟體家族分類 / Malware family classification")
print(f"• 資料譜系追蹤 / Data lineage tracking")
print(f"• 合規性審計支援 / Compliance audit support")